In [1]:
import json
import pyspark
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
from joblib import dump, load
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler, StandardScaler 
from pyspark.sql.functions import from_utc_timestamp

In [2]:
spark = (SparkSession
         .builder
         .master('spark://192.168.100.38:7077')
         .appName('MalwareDetection')
         .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
         .config("spark.mongodb.input.uri","mongodb://ta:ta@192.168.100.29:27018/MalwareDetection.data?authSource=admin")
         .config("spark.mongodb.output.uri","mongodb://ta:ta@192.168.100.29:27018/MalwareDetection.data?authSource=admin")
         .getOrCreate())

spark.conf.set("spark.sql.caseSensitive", "true")

sc = spark.sparkContext

In [3]:
model = "/media/kmdr7/Seagate/TA/MODELS/ModelWithoutScaler/RandomForest.joblib"

In [ ]:
schema = spark.read.load("hdfs://lab-1:9000/schema/schema.json", format="json")

In [ ]:
raw = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "192.168.100.29:19092").option("subscribe", "netflow_formatted").option("startingOffsets", "latest").load()

In [ ]:
parsed = raw.selectExpr("cast (value as string) as json").select(F.from_json("json",schema.schema).alias("data")).select("data.*")

In [ ]:
parsed.schema

In [ ]:
# stream = raw.writeStream.foreachBatch(lambda df, epoch_id: print(df, epoch_id)).start()
# stream.awaitTermination()